In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os
from dotenv import load_dotenv 
load_dotenv()
# GLOBAL CONFIGURATION
class CFG:
    # POSTGRES 
    OLLAMA_SERVER_URL=os.getenv("OLLAMA_SERVER_URL")
    DB_NAME=os.getenv("DATABASE_NAME")
    TABLE_NAME = os.getenv("VECTOR_TABLE_NAME")
    CUSTOM_TABLE_NAME="custom_theme_collection"
    PORT=os.getenv("PORT")
    CONNECTION_STRING = os.getenv("VECTOR_DATABASE_URL")

    # DIRECTORY
    EXTRACTED_FEATURE_DIR="./dataset/extracted_features"
    METADATA_DIR="./dataset/metadata"
    CACHE_FOLDER=".cache/huggingface/hub/"
    
    # EMBEDDING MODEL
    EMBEDDING_MODEL_NAME="BAAI/bge-small-en-v1.5"

    # REDIS
    REDIS_URL=os.getenv("REDIS_URL")
    REDIS_HOST=os.getenv("REDIS_HOST")
    REDIS_USERNAME=os.getenv("REDIS_USERNAME")
    REDIS_PASSWORD=os.getenv("REDIS_PASSWORD")

### 1. Setup Embedding Configurations

In [3]:
# Setup LLM using Ollama
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3", request_timeout=120.0, base_url=CFG.OLLAMA_SERVER_URL)

In [4]:
# Setup Embedding Model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", cache_folder=".cache/huggingface/hub/")

# Define global settings embedding model
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

/home/atuan/Desktop/chest-xray-disease-multi-classification/be-rag/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Load 

#### 2.1 Define PgVector store


In [5]:
import psycopg2

conn = psycopg2.connect(CFG.CONNECTION_STRING)
conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {CFG.DB_NAME}")
#     c.execute(f"CREATE DATABASE {CFG.DB_NAME}")

In [6]:
from sqlalchemy import make_url
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.storage.docstore.postgres import PostgresDocumentStore

url = make_url(CFG.CONNECTION_STRING)

vector_store = PGVectorStore.from_params(
    database=CFG.DB_NAME,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=CFG.TABLE_NAME,
    embed_dim=384,  # change to our 768 if using base model
    hnsw_kwargs={
        "hnsw_m": 16,
        "hnsw_ef_construction": 64,
        "hnsw_ef_search": 40,
        "hnsw_dist_method": "vector_cosine_ops",
    },
)


docstore = PostgresDocumentStore.from_params(
    database=CFG.DB_NAME,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="docstore",
)


storage_context = StorageContext.from_defaults(vector_store=vector_store)

#### 2.2 Load metadata & extracted features


##### 2.2.1 Create new emmbeding

In [7]:
from llama_index.core import (
    StorageContext,
    Settings,
    SimpleDirectoryReader
)

def load_data(path_to_docs='../data/documents/'):
    # Load PDF documents
    required_exts = [".pdf"]
    docs = SimpleDirectoryReader(
        input_dir=path_to_docs,   
        required_exts=required_exts,
        recursive=True,   
        filename_as_id=True 
    ).load_data()
    return docs

In [8]:
docs = load_data()
print(f"Loaded {len(docs)} documents")

Loaded 153 documents


In [13]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(docs)
print(f"{len(nodes)} nodes created from documents")

230 nodes created from documents


In [14]:
from llama_index.core.ingestion import (
    DocstoreStrategy,
    IngestionPipeline,
    IngestionCache
)
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from redis import Redis


# Create a Redis client with authentication
redis_client = Redis.from_url(CFG.REDIS_URL)

cache_store = IngestionCache(
    cache=RedisCache.from_redis_client(redis_client),
    collection="theme_ingestion_cache",
)

pipeline = IngestionPipeline(
    transformations=[
        # TokenTextSplitter(),
        embed_model,
    ],
    vector_store=vector_store,
    cache=cache_store,
    docstore=docstore,
    docstore_strategy=DocstoreStrategy.UPSERTS_AND_DELETE  # others strategy: DUPLICATES_ONLY, UPSERTS_AND_DELETE
)


Could not create async redis client from sync client, pass in `async_redis_client` explicitly.


In [15]:

def ingest_docs(pipeline, docs):
    nodes = pipeline.run(documents=docs, show_progress=True)
    print(f"Ingested {len(nodes)} Nodes")
    return nodes
nodes = ingest_docs(pipeline, docs)
print(f"Stored {len(nodes)} nodes in the vector store")

Generating embeddings: 0it [00:00, ?it/s]

Ingested 0 Nodes
Stored 0 nodes in the vector store


In [ ]:
# https://docs.llamaindex.ai/en/stable/examples/vector_stores/postgres/#create-the-index

# If a dimension mismatch error occurs during embedding insertion, 
# ensure the database column's vector type matches the dimensions of the embeddings generated by the model.
# ex: BAAI/bge-small-en-v1.51 (dim: 384)
# ex: BAAI/bge-base-en-v1.51 (dim: 768)
# Use the following SQL command to update the column type to match the expected dimensions:
# 
# ALTER TABLE public.data_theme_colection ALTER COLUMN embedding TYPE VECTOR(768);
# 
# Note: Replace "768" with the appropriate dimension size based on the embedding model being used.
# Ensure that all existing embeddings in the column are updated accordingly to avoid inconsistency.

# index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, show_progress=True)
# query_engine = index.as_query_engine()

Parsing nodes:   0%|          | 0/153 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 230/230 [00:03<00:00, 74.13it/s]


##### 2.2.2 Or load from existing vector database

In [16]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    pipeline.vector_store, embed_model=embed_model
)

# Querying

In [17]:
def query_chest_documents(query_engine, user_query: str):
    '''Query chest disease documents and return response in Vietnamese'''
    format_prompt = f"""
    Dựa trên tài liệu về bệnh lý ngực, hãy cung cấp thông tin về: "{user_query}"
    
    Yêu cầu:
    - Trả lời bằng tiếng Việt
    - Thông tin phải chính xác và dựa trên tài liệu
    - Nêu rõ nguồn tài liệu nếu có
    - Cung cấp thông tin đầy đủ nhưng súc tích
    """
    response = query_engine.query(format_prompt)
    print(f"Câu hỏi: {user_query}")
    print(f"Trả lời: {response}")


In [19]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    response_mode="tree_summarize"
)

# Example queries
query_chest_documents(query_engine, "Triệu chứng của viêm phổi")
print("-"*100)
query_chest_documents(query_engine, "Phương pháp chẩn đoán ung thư phổi")
print("-"*100)
query_chest_documents(query_engine, "Cách điều trị bệnh lao phổi")

Câu hỏi: Triệu chứng của viêm phổi
Trả lời: Theo các tài liệu bệnh lý ngực, triệu chứng của viêm phổi bao gồm:

* Sốt cao
* Rét run (rash)
* Ho khạc đờm mủ (coughing up mucus)
* Đau ngực kiểu màng phổi (chest pain)

(Nguồn: Phac-do-VP-cong-dong-nguoi-lon_2020.pdf, trang 27-29)
----------------------------------------------------------------------------------------------------
Câu hỏi: Phương pháp chẩn đoán ung thư phổi
Trả lời: Phương pháp chẩn đoán ung thư phổi bao gồm:

* X-quang phổi: giúp phát hiện đám mờ trên phổi, là một triệu chứng của ung thư phổi.
* Chụp cắt lớp vi tính (CT) ngực: giúp chụp ảnh phổi và các bộ phận xung quanh để phát hiện vị trí ung thư phổi.
* Sinh thiết phế quản: giúp lấy mẫu mô từ phế quan để chẩn đoán ung thư phổi.
* Vi sinh học: giúp tìm kiếm tế bào ung thư trong máu hoặc dịch phế quản.

Theo tài liệu "Phac-do-VP-cong-dong-nguoi-lon_2020.pdf" (trang 31-33), phương pháp chẩn đoán ung thư phổi bao gồm các bước sau:

1. X-quang phổi: giúp phát hiện đám mờ trê

In [20]:
query_chest_documents(query_engine,"Liệt kê Tiêu chuẩn nhập khoa Điều trị tích cực của ATS")

Câu hỏi: Liệt kê Tiêu chuẩn nhập khoa Điều trị tích cực của ATS
Trả lời: Theo các tiêu chuẩn của American Thoracic Society (ATS), Tiêu chuẩn nhập khoa Điều trị tích cực bao gồm hai tiêu chuẩn chính và chín tiêu chuẩn phụ.

Tiêu chuẩn chính:

1. Suy hô hấp cần phải thông khí cơ học.
2. Sốc nhiễm khuẩn cần phải dùng thuốc vận mạch.

Tiêu chuẩn phụ:

1. Tần số thở > 30 lần/phút.
2. PaO2/FiO2 < 250.
3. T ổn thƣơng nhiều thùy phổi trên phim X-quang.
4. Lú lẫn, mất định hướng.
5. Ure máu (BUN > 20 mg/dL).
6. Bạch cầu máu < 4000/mm3.
7. Giảm tiểu cầu (< 100.000/mm3).
8. Hạ thân nhiệt (< 36oC).
9. Hạ huyết áp cần phải bù dịch tích cực.

Nguồn tài liệu: Theo tài liệu về bệnh lý ngực, Tiêu chuẩn nhập khoa Điều trị tích cực của ATS bao gồm các chỉ số sau... (trang 40).


In [21]:
!pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
